In [3]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import sys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import sys
from scipy import signal

resampling = 'resampled'
depth_list = ['GM']
nruns = 4
subject = 'sub-46'
session = '02'
task = 'RET'


# For me it is only one depth = GM 
for depth in range(len(depth_list)):
    proc_tc_RH = [] # Store the time course form the Right Hemisphere
    proc_tc_LH = [] # Store the time course form the Left Hemisphere

    for run in range(nruns): # Approximately 4 runs 
        # Load GIFTI files for left and right hemispheres
        proc_tc_L = nib.load(
            f'{MAIN_PATH}/{resampling}/{subject}/ses-{session}/{denoising}/{subject}_ses-{session}_task-{task}_run-{run + 1}_space-fsnative_hemi-L_desc-{denoising}_bold_GM.gii')
        proc_tc_R = nib.load(
            f'{MAIN_PATH}/{resampling}/{subject}/ses-{session}/{denoising}/{subject}_ses-{session}_task-{task}_run-{run + 1}_space-fsnative_hemi-R_desc-{denoising}_bold_GM.gii')

        # Aggregate the data for the hemisphere separately 
        tc_R = proc_tc_R.agg_data().T  # (time points, vertices)
        tc_L = proc_tc_L.agg_data().T  # (time points, vertices)
        print(tc_R.shape)
        # Not needed now 
        #if tc_R.shape[0] > 136:
        #    tc_R = tc_R[:136, :]
        #    print(tc_R.shape)
        #if tc_L.shape[0] > 136:
        #    tc_L = tc_L[:136, :]

        # Normalize 
        tc_m_R = tc_R * np.expand_dims(np.nan_to_num((100 / np.mean(tc_R, axis=0))), axis=0)
        tc_m_L = tc_L * np.expand_dims(np.nan_to_num((100 / np.mean(tc_L, axis=0))), axis=0)

        # Baseline correction
        baseline_R = np.median(tc_m_R[:5], axis=0)
        baseline_L = np.median(tc_m_L[:5], axis=0)

        tc_m_R = tc_m_R - baseline_R
        tc_m_L = tc_m_L - baseline_L

        # Filter if enabled
        if filter == 1:
            mean_R = np.mean(tc_m_R, axis=0)
            mean_L = np.mean(tc_m_L, axis=0)

            tc_m_R = signal.detrend(tc_m_R, axis=0) + mean_R
            tc_m_L = signal.detrend(tc_m_L, axis=0) + mean_L

            # Highpass-filtering
            TR = 1.5
            fs = 1 / TR
            lowcut = 0.006
            nyquist = 0.5 * fs
            f_low = lowcut / nyquist
            sos = signal.butter(8, [f_low], 'highpass', fs=fs, output='sos')
            tc_m_R = signal.sosfiltfilt(sos, tc_m_R, axis=0)
            tc_m_L = signal.sosfiltfilt(sos, tc_m_L, axis=0)

        # Store processed data for saving
        proc_tc_RH.append(tc_m_R)
        proc_tc_LH.append(tc_m_L)

    # Compute median time course across runs
    mean_proc_tc_RH = np.median(np.array(proc_tc_RH), axis=0)
    mean_proc_tc_LH = np.median(np.array(proc_tc_LH), axis=0)

    # Save the processed time courses for RH and LH separately
    path = f'{MAIN_PATH}/pRFM/{subject}/ses-{session}/{denoising}/'
    os.makedirs(path, exist_ok=True)
    np.save(f'{path}{subject}_ses-{session}_task-{task}_hemi-rh_desc-avg_bold_GM.npy', mean_proc_tc_RH)
    np.save(f'{path}{subject}_ses-{session}_task-{task}_hemi-lh_desc-avg_bold_GM.npy', mean_proc_tc_LH)

In [4]:
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
subject = 'sub-46'
session = '02'
denoising = 'nordic'
depth_list = 'GM'
nruns = 4
resampling = 'resampled'
task = 'RET'

for depth in range(depth_list.__len__()):
    proc_tc_L = []
    proc_tc_R = []
    proc_tc = []
    for run in range(nruns):
        proc_tc_L = nib.load(
            f'{MAIN_PATH}/{resampling}/{subject}/ses-{session}/{denoising}/{subject}_ses-{session}_task-{task}_run-{run + 1}_space-fsnative_hemi-L_desc-{denoising}_bold_GM.gii')
        proc_tc_R = nib.load(
            f'{MAIN_PATH}/{resampling}/{subject}/ses-{session}/{denoising}/{subject}_ses-{session}_task-{task}_run-{run + 1}_space-fsnative_hemi-R_desc-{denoising}_bold_GM.gii')
        tc = np.vstack([proc_tc_L.agg_data(), proc_tc_R.agg_data()]).T
        if tc.shape[0]>136:
            tc=tc[:,:] 
        tc=tc[:,:] 
        tc_m = tc * np.expand_dims(np.nan_to_num((100 / np.mean(tc, axis=0))), axis=0) 

        # Baseline correction
        baseline = np.median(tc_m[:5], axis=0)
        tc_m = tc_m - baseline # ORIGINAL TIME COURSE

        if filter==1:
            from scipy import signal
            mean=np.mean(tc_m, axis=0) # Remove linear trend without demeaning
            tc_m=signal.detrend(tc_m, axis=0)+mean
            # Highpass-filtering
            TR = 1.5                                                            ## TR is the time between successive MRI scans, measured in seconds.                              
            fs = 1 / TR  # Hz                                                   ## FS is the sampling frequency, which is the number of samples per second. It's the inverse of the TR.
            lowcut=0.006 # cut-off freq of the filter                            ## Lowcut and highcut define the frequency range for the high-pass filter. Only the lowcut frequency is used.
            highcut=0.015 # cut-off freq of the filter
            nyquist=0.5*fs                                                      ## Nyquist frequency is half the sampling frequency.
            f_low = lowcut/nyquist;                                             ## The cut-off frequencies are normalized. (e.i 0,02 )
            f_high = highcut/nyquist;
            sos = signal.butter(8, [f_low],'highpass', fs=fs,output='sos')      ## Azzurra used low-pass 4th order Butterworth filter with a cut-off frequency of 0.1 Hz
            tc_m = signal.sosfiltfilt(sos, tc_m, axis=0)

        proc_tc.append(tc_m)
    mean_proc_tc = np.median(np.array(proc_tc), axis=0)
    psc = (mean_proc_tc) # FILTERED TIME COURSE

    path=f'{MAIN_PATH}/pRFM/{subject}/ses-{session}/{denoising}/'
    os.makedirs(path, exist_ok=True)
    np.save(f'{MAIN_PATH}/pRFM/{subject}/ses-{session}/{denoising}/{subject}_ses-{session}_task-{task}_hemi-LR_desc-avg_bold_{depth_list[depth]}_trial.npy',psc)

/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_63170/1011283980.py:23: RuntimeWarning: divide by zero encountered in divide
  tc_m = tc * np.expand_dims(np.nan_to_num((100 / np.mean(tc, axis=0))), axis=0)
